In [2]:
import mysql.connector
from mysql.connector import Error
import re # Regular Expression
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory # Sastrawi Stemmer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory # Sastrawi Stopword Remover
from gensim import corpora, models, similarities

In [3]:
db_config = {
        "host": "mysql-students-my-student-projects.h.aivencloud.com",
        "port": "13993",
        "user": "galangarsandy",
        "password": "AVNS_1coj2um74BOgK_AGXaI",
        "database": "quran"
    }
class DatabaseConnection:
    def __init__(self):
        try:
            self.connection = mysql.connector.connect(**db_config)
            self.cursor = self.connection.cursor()
            print("Connected to the database\n")
        except Error as err:
            print(f"The error '{err}' occurred")
            
    def Select(self, sql):
        try:
            self.cursor.execute(sql)
            return self.cursor.fetchall()
        except Error as err:
            print(f"Select data ERROR: ", err)
    
    def __del__(self):
        self.connection.close()  

db = DatabaseConnection() # Membuat objek database connection
data_quran = db.Select("SELECT * FROM id_indonesian limit 100")

Connected to the database



In [4]:
stemmer = StemmerFactory().create_stemmer() # Membuat objek stemmer
stopwords = StopWordRemoverFactory().create_stop_word_remover() # Membuat objek stopwords

corpus = [] # List untuk menampung data quran yang sudah di pre-processing

def pre_processing(text):
    pre_process = re.sub('[^a-zA-Z]', ' ', text.lower()) # Menghilaangkan karakter selain huruf a-z dan A-Z dan diubah menjadi spasi. Lalu diubah menjadi huruf kecil
    pre_process = re.sub(' +', ' ', pre_process) # Menghilangkan spasi yang berlebihan
    pre_process = stopwords.remove(pre_process) # Menghilangkan stopwords atau kata-kata yang tidak penting
    pre_process = stemmer.stem(pre_process) # Melakukan stemming atau mengubah kata-kata menjadi kata dasar
    return pre_process

for (index, surat, ayat, text) in data_quran:
    pre_process_data = pre_processing(text)
    corpus.append((index, surat, ayat, pre_process_data)) # Menambahkan data ke dalam list corpus

In [14]:
# Pra-pemrosesan teks
tokenized_documents = [doc.split() for (index, sura, aya, doc) in corpus] # membuat tokenisasi dari data corpus, atau memecah menjadi per kata
dictionary = corpora.Dictionary(tokenized_documents) # membuat dictionary dari tokenisasi, jadi tidak ada data atau token yang duplikat

"""
Di bawah ini, doc2bow mengubah setiap dokumen yang sudah ditokenisasi menjadi representasi BoW. 
Dalam representasi ini, setiap dokumen direpresentasikan sebagai daftar tuple, di mana setiap tuple terdiri dari (ID kata, frekuensi kata dalam dokumen).
"""
corpusx = [dictionary.doc2bow(doc) for doc in tokenized_documents]

print(corpusx)
print(dictionary)

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1)], [(0, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)], [(1, 2), (2, 1), (4, 1)], [(11, 1), (12, 1), (13, 1)], [(14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)], [(20, 1), (21, 1), (22, 1)], [(14, 1), (20, 3), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 2)], [(32, 1), (33, 1), (34, 1)], [(22, 1), (25, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1)], [(16, 1), (25, 2), (31, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1)], [(25, 2), (30, 2), (35, 1), (36, 3), (38, 1), (45, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 2), (56, 1)], [(10, 1), (22, 1), (25, 1), (28, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1)], [(23, 2), (28, 2), (41, 2), (45, 1), (62, 1), (63, 1), (64, 2), (65, 1), (66, 2), (67, 1), (68, 1)], [(0, 1), (57, 1), (63, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1)], [(0, 1), (12, 1), (24, 1), (28, 

In [6]:
"""
models.LsiModel adalah kelas dalam Gensim yang digunakan untuk menerapkan Latent Semantic Indexing. 
LSI adalah teknik yang digunakan untuk menemukan pola semantik dalam data teks dengan mengurangi dimensi dari representasi yang ada.
"""

# Menerapkan LSI
lsi = models.LsiModel(corpusx, id2word=dictionary, num_topics=50)

In [21]:
input_query = input("Inputkan Keywords: ")

query_bow = dictionary.doc2bow(input_query.split()) # Membuat Representasi BoW untuk Query

"""
Di sini, representasi BoW dari query (query_bow) diterapkan pada model LSI (lsi) untuk mendapatkan representasi dalam ruang topik yang lebih rendah.
Dengan cara ini, Anda akan mendapatkan representasi LSI dari query, yang memungkinkan Anda untuk menginterpretasikan query dalam konteks topik yang telah diidentifikasi oleh model LSI.
query_lsi sekarang berisi informasi tentang bagaimana query tersebut berkaitan dengan topik-topik yang telah ditemukan sebelumnya dalam corpus.
"""
query_lsi = lsi[query_bow]

[(27, 1), (194, 1), (200, 1)]
[(0, 0.036246102215289656), (1, -0.005783334322419764), (2, 0.15945681538429302), (3, 0.1416916511580063), (4, -0.022888544623904696), (5, 0.34245162202220447), (6, -0.01833997003576182), (7, -0.2011601365175352), (8, 0.12249140466450154), (9, 0.10791464525158731), (10, 0.026397132737603742), (11, 0.05421612256635211), (12, 0.16732031528732633), (13, -0.06389930547566514), (14, -0.1455180573523751), (15, -0.12664612433146144), (16, -0.021296386077575273), (17, -0.08095880693238885), (18, -0.049094146482627665), (19, -0.09157075260697461), (20, -0.0006954818400297311), (21, -0.0002646171848711132), (22, 0.13493803715092015), (23, 0.06844138328813806), (24, -0.05210884011486001), (25, -0.09537251585445575), (26, -0.04923558519232625), (27, 0.0208368112943937), (28, -0.09764487155842477), (29, 0.08137169383216834), (30, -0.16862639135606136), (31, 0.00867519326417543), (32, -0.007674686353069199), (33, -0.009779333925375014), (34, -0.05100430312026115), (35, 

In [33]:
"""
similarities.MatrixSimilarity adalah fungsi dari Gensim yang digunakan untuk membangun indeks kemiripan berbasis matriks. 
Ini memungkinkan Anda untuk dengan cepat menghitung kemiripan antara dokumen dan query dalam ruang topik LSI.
"""
index = similarities.MatrixSimilarity(lsi[corpusx])

"""
Dalam kode dibawah ini, index adalah objek indeks kemiripan yang telah Anda buat sebelumnya dengan menggunakan similarities.MatrixSimilarity. Objek ini menyimpan representasi LSI dari semua dokumen dalam korpus.
query_lsi adalah representasi LSI dari query yang Anda masukkan sebelumnya.
"""
sims = index[query_lsi]

"""
dalam kode dibawah, enumerate(sims) menggabungkan indeks dokumen dengan nilai kemiripan dari sims. Hasilnya adalah list tuple di mana setiap tuple berisi (indeks dokumen, nilai kemiripan).
sorted(..., key=lambda item: -item[1]) mengurutkan list berdasarkan nilai kemiripan secara menurun (dari yang tertinggi ke terendah). Tanda minus (-) digunakan untuk membalikkan urutan, sehingga dokumen dengan kemiripan tertinggi muncul terlebih dahulu.
"""
ranked_documents = sorted(enumerate(sims), key=lambda item: -item[1]) #Pengurutan Hasil Kemiripan

In [29]:
print("\nHasil Temu Balik Informasi:\n")
for rank, (doc_id, score) in enumerate(ranked_documents):
    print(f"Dokumen {rank + 1}: {data_quran[doc_id]} (Similarity: {score:.4f})\n")
    if(rank == 2):
        break


Hasil Temu Balik Informasi:

Dokumen 1: (32, 2, 25, 'Dan sampaikanlah berita gembira kepada mereka yang beriman dan berbuat baik, bahwa bagi mereka disediakan surga-surga yang mengalir sungai-sungai di dalamnya. Setiap mereka diberi rezeki buah-buahan dalam surga-surga itu, mereka mengatakan: "Inilah yang pernah diberikan kepada kami dahulu". Mereka diberi buah-buahan yang serupa dan untuk mereka di dalamnya ada isteri-isteri yang suci dan mereka kekal di dalamnya.') (Similarity: 0.6849)

Dokumen 2: (88, 2, 81, '(Bukan demikian), yang benar: barangsiapa berbuat dosa dan ia telah diliputi oleh dosanya, mereka itulah penghuni neraka, mereka kekal di dalamnya.') (Similarity: 0.4652)

Dokumen 3: (89, 2, 82, 'Dan orang-orang yang beriman serta beramal saleh, mereka itu penghuni surga; mereka kekal di dalamnya.') (Similarity: 0.4598)



#### Tugas No 2

In [15]:
def rankingkan(query_lsi):
    index = similarities.MatrixSimilarity(lsi[corpusx])

    sims = index[query_lsi]

    ranked_documents = sorted(enumerate(sims), key=lambda item: -item[1])

    return ranked_documents

In [16]:
input_query = input("Inputkan Keywords: ")
# print(input_query)

query_preproc = pre_processing(input_query)
# print(query_preproc)

query_bow = dictionary.doc2bow(query_preproc.split()) # Membuat Representasi BoW untuk Query

query_lsi = lsi[query_bow]

ranked_documents = rankingkan(query_lsi)

print(f'Query Anda: "{input_query}"')
print("Hasil Temu Balik Informasi:\n")
for rank, (doc_id, score) in enumerate(ranked_documents):
    print(f"Dokumen {rank + 1}: {data_quran[doc_id]} (Similarity: {score:.4f})\n")
    if(rank == 2):
        break

Query Anda: "ibadah Zakat, Shalat & Zakat"
Hasil Temu Balik Informasi:

Dokumen 1: (90, 2, 83, 'Dan (ingatlah), ketika Kami mengambil janji dari Bani Israil (yaitu): Janganlah kamu menyembah selain Allah, dan berbuat kebaikanlah kepada ibu bapa, kaum kerabat, anak-anak yatim, dan orang-orang miskin, serta ucapkanlah kata-kata yang baik kepada manusia, dirikanlah shalat dan tunaikanlah zakat. Kemudian kamu tidak memenuhi janji itu, kecuali sebahagian kecil daripada kamu, dan kamu selalu berpaling.') (Similarity: 0.6179)

Dokumen 2: (50, 2, 43, "Dan dirikanlah shalat, tunaikanlah zakat dan ruku'lah beserta orang-orang yang ruku'.") (Similarity: 0.5184)

Dokumen 3: (10, 2, 3, '(yaitu) mereka yang beriman kepada yang ghaib, yang mendirikan shalat, dan menafkahkan sebahagian rezeki yang Kami anugerahkan kepada mereka.') (Similarity: 0.2915)

